In [1]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [17]:
currdir= os.getcwd()
input_directory = os.path.join(currdir, 'Training_WFDB')
output_directory = os.path.join(currdir, 'Mel_data')    
if not os.path.isdir(input_directory):
        os.mkdir(input_directory)
if not os.path.isdir(output_directory):
        os.mkdir(output_directory)        

In [18]:
# Find files
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)

In [19]:
input_files

['A3066.mat',
 'A4872.mat',
 'A2971.mat',
 'A4463.mat',
 'A1499.mat',
 'A4135.mat',
 'A0099.mat',
 'A5945.mat',
 'A0101.mat',
 'A6165.mat',
 'A6623.mat',
 'A1510.mat',
 'A5206.mat',
 'A6861.mat',
 'A1695.mat',
 'A4785.mat',
 'A5838.mat',
 'A5764.mat',
 'A0736.mat',
 'A4505.mat',
 'A2486.mat',
 'A4292.mat',
 'A1249.mat',
 'A2787.mat',
 'A3112.mat',
 'A2502.mat',
 'A0561.mat',
 'A0354.mat',
 'A3610.mat',
 'A1095.mat',
 'A1237.mat',
 'A0290.mat',
 'A3675.mat',
 'A1245.mat',
 'A2951.mat',
 'A2442.mat',
 'A2273.mat',
 'A4847.mat',
 'A5914.mat',
 'A2457.mat',
 'A4652.mat',
 'A3839.mat',
 'A5344.mat',
 'A3325.mat',
 'A5149.mat',
 'A1147.mat',
 'A4396.mat',
 'A4677.mat',
 'A3641.mat',
 'A3509.mat',
 'A5180.mat',
 'A1654.mat',
 'A6139.mat',
 'A6663.mat',
 'A5743.mat',
 'A4251.mat',
 'A3508.mat',
 'A2306.mat',
 'A2458.mat',
 'A3184.mat',
 'A3193.mat',
 'A2542.mat',
 'A5859.mat',
 'A0768.mat',
 'A5758.mat',
 'A0008.mat',
 'A5539.mat',
 'A1090.mat',
 'A4053.mat',
 'A2758.mat',
 'A3473.mat',
 'A275

In [20]:
np.shape(input_files)

(6877,)

In [37]:
n_mels = 128
frame_length = 0.025 # 1초를 40개로?
hop_length = 512
#hop_length = 512 -> 얘가 x축 bin 결정
#input_nfft = int(round(sr*frame_length))
n_fft = 1024
sr = 500
def Mel(data, sr, n_fft, hop_length, n_mels):
    S = librosa.feature.melspectrogram(data, sr=sr, n_fft=n_fft, 
                                       hop_length=hop_length, 
                                       n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return S

In [34]:
def get_12ECG_features(header_data):
    tmp_hea = header_data[0].split(' ')
    ptID = tmp_hea[0]
    num_leads = int(tmp_hea[1])
    sample_Fs= int(tmp_hea[2])
    gain_lead = np.zeros(num_leads)
    for ii in range(num_leads):
        tmp_hea = header_data[ii+1].split(' ') # +1인 이유는 첫 행을 빼야하니
        gain_lead[ii] = int(tmp_hea[2].split('/')[0]) # 아마도 모든 행을 1000mV로 채울듯

    for iline in header_data:
        if iline.startswith('#Age'):
            tmp_age = iline.split(': ')[1].strip()
            age = int(tmp_age if tmp_age != 'NaN' else 57)
        elif iline.startswith('#Sex'):
            tmp_sex = iline.split(': ')[1]
            if tmp_sex.strip()=='Female':
                sex =1
            else:
                sex=0
        elif iline.startswith('#Dx'):
            label = iline.split(': ')[1].split(',')[0]
    return ptID, num_leads, sample_Fs, gain_lead, age, sex, label

In [38]:
for i in range(len(input_files)): # loops over participants
    filename = os.path.join(input_directory, input_files[i])
    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)
    new_file = filename.replace('.mat', '.hea')
    input_header_file = os.path.join(new_file)
    with open(input_header_file,'r') as f:
        header_data=f.readlines()
    ptID, num_leads, sample_Fs, gain_lead, age, sex, label = get_12ECG_features(header_data)
    S = Mel(data, sr, n_fft, hop_length, n_mels)
    

ParameterError: Invalid shape for monophonic audio: ndim=2, shape=(12, 25197)

In [39]:
ptID

'A3066'

In [40]:
np.shape(data)

(12, 25197)